In [1]:
import pandas as pd 
import scipy.stats as st
import numpy as np


# Mostrar todo el contenido de las celdas (sin recortes)
pd.set_option('display.max_colwidth', None)

# Mostrar todas las columnas 
pd.set_option('display.max_columns', None)

In [2]:
ruta = "/home/donsson/proyectos/MODELO ABASTECIMIENTO/csvsalidas/clasificacion_mia2025.csv"

dp = pd.read_csv(ruta)

In [3]:
dp = dp.drop(columns=["Unnamed: 0"])

dp["store_name"] = dp["store_name"].replace({"PRINCIPAL COTA":"MOSTRADOR COTA"})

In [4]:
stock_seguridad = {
    "SUCURSAL CALLE 6": 50.00,
    "SUCURSAL VALLADOLID": 50.00,
    "SUCURSAL NORTE": 50.00,
    "SUCURSAL MEDELLIN": 150.00,
    "MOSTRADOR COTA": 300.00,
    "SUCURSAL BARRANQUILLA": 150.00,
    "SUCURSAL BUCARAMANGA": 150.00,
    "SUCURSAL CALI": 150.00,
   # "ANTIOQUEÑA DE LUBRICANTES SGP SAS": 300.00
}

reorden_ideal = {
    "SUCURSAL CALLE 6": {"Punto de reorden": 4.00, "Inventario ideal": 5.00},
    "SUCURSAL VALLADOLID": {"Punto de reorden": 4.00, "Inventario ideal": 5.00},
    "SUCURSAL NORTE": {"Punto de reorden": 4.00, "Inventario ideal": 5.00},
    "SUCURSAL MEDELLIN": {"Punto de reorden": 7.00, "Inventario ideal": 9.00},
    #"ANTIOQUEÑA DE LUBRICANTES SGP SAS": {"Punto de reorden": 5.00, "Inventario ideal": 7.00},
    "SUCURSAL BARRANQUILLA": {"Punto de reorden": 7.00, "Inventario ideal": 9.00},
    "SUCURSAL BUCARAMANGA": {"Punto de reorden": 7.00, "Inventario ideal": 9.00},
    "MOSTRADOR COTA": {"Punto de reorden": 2.50, "Inventario ideal": 3.00},
    "SUCURSAL CALI": {"Punto de reorden": 7.00, "Inventario ideal": 9.00}, # Tiene para 7 semnaas de inventario , y cubre el inventario 9 semanas
}


In [5]:
dp.head(10)

,store_name,product_ref,Año,semana,Ema,Costo unitario,Venta costo EMA,Venta costo %,Acumulado costo,Desviacion EMA,Desviacion EMA %,acumulado_cantidad,coef_ventas,acumulado_desviacion,acumulado_desviacion_p,acumulado_combinado,Clasificacion,Nivel de servicio
0,MOSTRADOR COTA,DCS10536137,2025.0,31,31.662640,51663.09,1635789.82,0.0093,0.0093,2.126527e+05,0.13,0.01,2.950820,0.13,0.8,0.19,AAA,0.999
1,MOSTRADOR COTA,DCS10536137,2025.0,28,29.493333,51663.09,1523716.72,0.0086,0.0179,0.000000e+00,0.00,0.01,0.000000,0.13,0.0,0.01,AAA,0.999
2,MOSTRADOR COTA,DCS10536137,2025.0,32,28.496376,51663.09,1472210.84,0.0084,0.0263,1.766653e+05,0.12,0.02,2.162162,0.25,0.8,0.20,AAA,0.999
3,MOSTRADOR COTA,DCS10536137,2025.0,29,26.744000,51663.09,1381677.68,0.0078,0.0341,0.000000e+00,0.00,0.02,0.000000,0.25,0.0,0.02,AAA,0.999
4,MOSTRADOR COTA,BCE00606125,2025.0,32,26.686976,20564.31,548799.25,0.0031,0.0992,7.585019e+11,1382111.72,0.03,0.000000,1382111.97,0.0,0.04,AAA,0.999
5,MOSTRADOR COTA,DCS10536137,2025.0,33,25.646738,51663.09,1324989.73,0.0075,0.0416,1.722487e+05,0.13,0.03,2.702703,1382112.10,0.8,0.21,AAA,0.999
6,MOSTRADOR COTA,DCS10536137,2025.0,30,24.069600,51663.09,1243509.91,0.0071,0.0487,7.461060e+04,0.06,0.04,1.934426,1382112.16,0.8,0.22,AAA,0.999
7,MOSTRADOR COTA,DCS10536137,2025.0,42,23.284502,51663.09,1202949.32,0.0068,0.0555,8.059760e+05,0.67,0.04,5.350318,1382112.83,1.0,0.26,AAA,0.999
8,MOSTRADOR COTA,BCE00606125,2025.0,33,21.349581,20564.31,439039.40,0.0025,0.1124,1.800062e+05,0.41,0.04,5.000000,1382113.24,1.0,0.27,AAA,0.999
9,MOSTRADOR COTA,DCS10536137,2025.0,34,20.517390,51663.09,1059991.77,0.0060,0.0615,1.589988e+05,0.15,0.05,3.243243,1382113.39,1.0,0.26,AAA,0.999


## INVENTARIO DE SEGURIDAD

In [6]:
# Copia del df original
df_std = dp.copy()

# Ordenar datos por tienda, producto, año y semana
df_std = df_std.sort_values(by=["store_name", "product_ref", "Año", "semana"])

# Calcular la desviación móvil (rolling) de EMA con ventana de 12 semanas
df_std["Desviacion_EMA_calc"] = (
    df_std.groupby(["store_name", "product_ref"])["Ema"]
    .transform(lambda x: x.rolling(window=12, min_periods=1).std())
)

# Calcular también la media móvil del EMA (opcional, para validación)
df_std["Ema_media"] = (
    df_std.groupby(["store_name", "product_ref"])["Ema"]
    .transform(lambda x: x.rolling(window=12, min_periods=1).mean())
)

# === INVENTARIO IDEAL y PUNTO DE REORDEN ===
# Diccionario con los dos parámetros (en semanas de cobertura)
reorden_ideal = {
    "SUCURSAL CALLE 6": {"Punto de reorden": 4.00, "Inventario ideal": 5.00},
    "SUCURSAL VALLADOLID": {"Punto de reorden": 4.00, "Inventario ideal": 5.00},
    "SUCURSAL NORTE": {"Punto de reorden": 4.00, "Inventario ideal": 5.00},
    "SUCURSAL MEDELLIN": {"Punto de reorden": 7.00, "Inventario ideal": 9.00},
    "SUCURSAL BARRANQUILLA": {"Punto de reorden": 7.00, "Inventario ideal": 9.00},
    "SUCURSAL BUCARAMANGA": {"Punto de reorden": 7.00, "Inventario ideal": 9.00},
    "MOSTRADOR COTA": {"Punto de reorden": 2.50, "Inventario ideal": 3.00},
    "SUCURSAL CALI": {"Punto de reorden": 7.00, "Inventario ideal": 9.00},
}

# Extraer columnas de reorden e ideal
df_std["lt_reorden"] = df_std["store_name"].map(lambda x: reorden_ideal.get(x, {}).get("Punto de reorden", np.nan))
df_std["lt_ideal"] = df_std["store_name"].map(lambda x: reorden_ideal.get(x, {}).get("Inventario ideal", np.nan))


# Calcular Z-score según nivel de servicio
df_std["Z"] = df_std["Nivel de servicio"].apply(lambda x: st.norm.ppf(x))

# === STOCK DE SEGURIDAD (rolling 12 semanas) ===
df_std["inventario_seguridad"] = (
    df_std["Z"] * df_std["Desviacion_EMA_calc"] * np.sqrt(df_std["lt_ideal"])
)

# Inventario ideal (nivel máximo recomendado)
df_std["inventario_ideal"] = (df_std["Ema"] * df_std["lt_ideal"]) + df_std["inventario_seguridad"]


# Punto de reorden (cuándo pedir)
df_std["punto_reorden"] = (df_std["Ema"] * df_std["lt_reorden"]) + df_std["inventario_seguridad"]



# === Redondear columnas numéricas ===
df_std["inventario_seguridad"] = df_std["inventario_seguridad"].round(2)
df_std["inventario_ideal"] = df_std["inventario_ideal"].round(2)
df_std["punto_reorden"] = df_std["punto_reorden"].round(2)




df_std = df_std.drop(columns=["Desviacion_EMA_calc","Ema_media","lt_reorden","lt_ideal","Z"])

# TARDA 2 MINUTOS 

In [7]:
df_std[(df_std["semana"]==42) & (df_std["store_name"]=="SUCURSAL BARRANQUILLA")].sort_values(by=["Ema"],ascending=False).head(10)

,store_name,product_ref,Año,semana,Ema,Costo unitario,Venta costo EMA,Venta costo %,Acumulado costo,Desviacion EMA,Desviacion EMA %,acumulado_cantidad,coef_ventas,acumulado_desviacion,acumulado_desviacion_p,acumulado_combinado,Clasificacion,Nivel de servicio,inventario_seguridad,inventario_ideal,punto_reorden
54008,SUCURSAL BARRANQUILLA,DAB02570025,2025.0,42,45.213202,13771.02,622631.91,0.0006,0.1571,118300.1,0.19,0.02,3.784343,2.20,1.0,0.30,AAA,0.999,80.82,487.74,397.31
54009,SUCURSAL BARRANQUILLA,DAB14570025,2025.0,42,45.131160,10117.70,456623.54,0.0005,0.2332,105023.4,0.23,0.02,3.589091,2.43,1.0,0.32,AAA,0.999,71.65,477.83,387.57
54056,SUCURSAL BARRANQUILLA,BCS00035125,2025.0,42,30.607635,20581.34,629946.14,0.0007,0.1505,62994.6,0.10,0.08,2.748858,8.68,0.8,0.29,AAA,0.999,32.31,307.78,246.56
54065,SUCURSAL BARRANQUILLA,BCS00025125,2025.0,42,28.850744,31057.73,896038.62,0.0009,0.0983,98564.2,0.11,0.08,1.960784,9.27,0.8,0.26,AAA,0.999,30.93,290.59,232.89
54070,SUCURSAL BARRANQUILLA,DAB02772025,2025.0,42,27.368237,10883.85,297871.79,0.0003,0.3403,50638.2,0.17,0.09,3.607565,9.95,1.0,0.40,AAA,0.999,51.73,298.05,243.31
54075,SUCURSAL BARRANQUILLA,BLS00037125,2025.0,42,26.673890,32494.89,866765.12,0.0009,0.1028,104011.8,0.12,0.09,1.617530,10.53,0.8,0.28,AAA,0.999,43.21,283.27,229.92
54109,SUCURSAL BARRANQUILLA,DAB02666025,2025.0,42,22.564250,11371.65,256592.75,0.0003,0.3739,30791.1,0.12,0.12,2.160772,2045930.10,0.8,0.38,AAA,0.999,18.62,221.70,176.57
54116,SUCURSAL BARRANQUILLA,BCS10035125,2025.0,42,21.896615,20902.34,457690.49,0.0005,0.2322,73230.5,0.16,0.13,2.536585,2045931.11,0.8,0.33,AAA,0.999,33.98,231.05,187.25
54128,SUCURSAL BARRANQUILLA,BCS00249125,2025.0,42,21.218834,27389.88,581181.32,0.0006,0.1685,87177.2,0.15,0.14,2.864686,2045937.15,0.8,0.32,AAA,0.999,31.98,222.95,180.52
54130,SUCURSAL BARRANQUILLA,DAB14772025,2025.0,42,21.160155,8825.90,186757.41,0.0002,0.4542,29881.2,0.16,0.14,3.453947,2045938.00,1.0,0.46,AAA,0.999,32.95,223.39,181.07


# INVENTARIO ACTUAL

In [8]:
ruta = "/home/donsson/proyectos/INVENTARIO/inventario_limpio.xlsx"

inv = pd.read_excel(ruta)

inv["product_ref"] = inv["Complete name"].str.extract(r"\[([A-Z0-9]+)\]")

In [9]:
inv.head(10)

,Sucursal,Estado,Codigo_interno,Complete name,Clase,Marca,categoria_producto,Cantidad,Costo unitario,Costo total,product_ref
0,PRINCIPAL COTA,Existencias,004 DA4207,[DAE04207004] DA4207 FILTRO AIRE SEGURIDAD KOMATSU (004 AF574SY),C,AUT*PARTS,Filtro,1.0,1.00,1.00,DAE04207004
1,PRINCIPAL COTA,Existencias,004 G314,[DLE00314004] G314 FILTRO ACEITE ESPECIAL (004 F62H),C,AUT*PARTS,Filtro,2.0,1890.00,3780.00,DLE00314004
2,PRINCIPAL COTA,Existencias,004 G401,[DLE00401004] G401 FILTRO ACEITE FIAT-ALLIS 75215486 (004 F75215486),C,AUT*PARTS,Filtro,7.0,1.00,7.00,DLE00401004
3,SUCURSAL MEDELLIN,Existencias,004 GA128,[DAE05128004] GA128 FILTRO AIRE FIAT DATSUN (004 GA128),C,AUT*PARTS,Filtro,2.0,1694.96,3389.92,DAE05128004
4,PRINCIPAL COTA,Existencias,004 GA128,[DAE05128004] GA128 FILTRO AIRE FIAT DATSUN (004 GA128),C,AUT*PARTS,Filtro,1.0,1694.96,1694.96,DAE05128004
5,PRINCIPAL COTA,Existencias,004 GA158,[DAE05158004] GA158 FILTRO AIRE RENULT 4 Y 6. (004 GA158),C,AUT*PARTS,Filtro,1.0,1722.50,1722.50,DAE05158004
6,PRINCIPAL COTA,Existencias,004 GA182,[DAE05182004] GA182 FILTRO AIRE SIMCA 1000 Y 1300 (004 GA182),C,AUT*PARTS,Filtro,1.0,1462.50,1462.50,DAE05182004
7,PRINCIPAL COTA,Existencias,004 GA321,[DAE05321004] GA321 FILTRO AIRE MONZA (004 GA321),C,AUT*PARTS,Filtro,2.0,1716.00,3432.00,DAE05321004
8,SUCURSAL BUCARAGAMNGA,Existencias,004 GS050,"[DLS00050004] GS050 FILTRO ACEITE SWIFT,SPRINT. (004 GS050)",C,AUT*PARTS,Filtro,1.0,6293.12,6293.12,DLS00050004
9,SUCURSAL NORTE,Existencias,004 GS050,"[DLS00050004] GS050 FILTRO ACEITE SWIFT,SPRINT. (004 GS050)",C,AUT*PARTS,Filtro,1.0,6293.12,6293.12,DLS00050004


In [10]:
inv["Cantidad"].sum() 

np.float64(311222.656)

# EVALUACION Y COMPARACION

In [11]:
import xmlrpc.client
import pandas as pd


# ===============================
# Conexión con Odoo
# ===============================
username = "juan.cano@donsson.com"   # tu usuario
password = "1000285668"              # tu contraseña
url = "https://donsson.com"          # URL del servidor
db = "Donsson_produccion"            # nombre de la base de datos

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")


#COSTO DE LOS PRODUCTOS

productos_ema = models.execute_kw(
    db, uid, password,
    "cs.analisis.costo", "search_read",[],
    #[
    #    [["semana","=",36]]
    #],
    {"fields": ["sucursal_id","product_name","ano","semana","ema","producto_costo_unitario","punto_reorden","clasificacion","almacen_id","inventario_seguridad","inventario_ideal" ]}
)

df = pd.DataFrame(productos_ema)

def extract_id(val):
    if isinstance(val, (list, tuple)) and len(val) > 0:
        return val[0]
    return None

def extract_name(val):
    if isinstance(val, (list, tuple)) and len(val) > 1:
        return val[1]
    return str(val)

df.rename(columns={"product_name": "product_ref"}, inplace=True)


df["store_id_num"] = df["sucursal_id"].apply(extract_id)
df["store_name"]   = df["sucursal_id"].apply(extract_name)

df["almacen_id_num"] = df["almacen_id"].apply(extract_id)
df["almacen_name"]   = df["almacen_id"].apply(extract_name)


In [12]:
df_41 = df_std[df_std["semana"]==41]

df_41["Clasificacion_mia"] = df_41["Clasificacion"]

/tmp/ipykernel_2556/2500382178.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_41["Clasificacion_mia"] = df_41["Clasificacion"]


In [13]:
# --- MERGE LIMPIO PARA COMPARAR ---
df_merge_simple = pd.merge(
    df_41[['store_name', 'product_ref', 'Clasificacion_mia', 
            'inventario_seguridad', 'inventario_ideal', 'punto_reorden']],
    df[['store_name', 'product_ref', 'clasificacion', 
         'inventario_seguridad', 'inventario_ideal', 'punto_reorden']],
    on=['store_name', 'product_ref'],
    how='inner',
    suffixes=('_std', '_real')
)

# --- COMPARACIÓN DE CLASIFICACIÓN ---
df_merge_simple['match_clasificacion'] = (
    df_merge_simple['Clasificacion_mia'] == df_merge_simple['clasificacion']
)

# --- OPCIONAL: ORDENAR PARA REVISAR FÁCIL ---
df_merge_simple = df_merge_simple.sort_values(by=['store_name', 'product_ref'])

# --- MOSTRAR SOLO LAS COLUMNAS RELEVANTES ---
cols = [
    'store_name', 'product_ref',
    'Clasificacion_mia', 'clasificacion', 'match_clasificacion',
    'inventario_seguridad_std', 'inventario_seguridad_real',
    'inventario_ideal_std', 'inventario_ideal_real',
    'punto_reorden_std', 'punto_reorden_real'
]

df_merge_simple = df_merge_simple[cols]



df_merge_simple.to_excel("/home/donsson/proyectos/MODELO ABASTECIMIENTO/csvsalidas/diferencias_ema.xlsx")

In [14]:
df_merge_simple.sample(10)

,store_name,product_ref,Clasificacion_mia,clasificacion,match_clasificacion,inventario_seguridad_std,inventario_seguridad_real,inventario_ideal_std,inventario_ideal_real,punto_reorden_std,punto_reorden_real
13330,SUCURSAL CALLE 6,DLE00401004,C,C,True,0.00,0.000000,0.00,0,0.00,0
11198,SUCURSAL CALLE 6,BHE01042125,C,C,True,0.00,0.000000,0.00,0,0.00,0
10330,SUCURSAL CALLE 6,BAC00188125,C,C,True,0.00,0.000000,0.00,0,0.00,0
13418,SUCURSAL CALLE 6,DLS00213189,B,B,True,2.01,2.135961,15.67,14,12.94,12
14416,SUCURSAL MEDELLIN,BAR02047125,C,C,True,0.00,0.000000,0.00,0,0.00,0
26450,SUCURSAL VALLADOLID,DCE00682115,C,C,True,0.00,0.000000,0.00,0,0.00,0
10858,SUCURSAL CALLE 6,BCS00387125,B,A,False,0.95,1.318871,7.64,7,6.30,6
17692,SUCURSAL MEDELLIN,DAE00964138,C,C,True,0.00,0.000000,0.00,0,0.00,0
26777,SUCURSAL VALLADOLID,DHE00308189,C,C,True,0.00,0.000000,0.00,0,0.00,0
22937,SUCURSAL NORTE,DAR09065025,C,B,False,0.00,0.371846,0.58,3,0.47,3
